# A FAIRE dans CNN_SL :
* chargement d'un modèle
* chargement des poids d'un modèle
* fonction eval_state

In [1]:
import keras
import go 
import CNN_policy

Using TensorFlow backend.


In [37]:
import numpy as np
# definition d'un joueur

class Player_rd(object): #joue au hasaer

    def __init__(self):
        return
    
    def get_move(self, state):
        # list with sensible moves
        sensible_moves = [move for move in state.get_legal_moves(include_eyes=False)]
        
      
        
        #parcourt les coups possible et ressort celui qu'aurai jouer le policy network    
        if len(sensible_moves) > 0:
            a=np.random.randint(0,len(sensible_moves),1) #on prend un coup au hasard
            return(sensible_moves[a[0]])
        

        # No 'sensible' moves available, so do pass move
        return go.PASS_MOVE

class Player_pl(object): # joue coup le plus probable donnée par le policy

    def __init__(self, policy_function):
        self.policy = policy_function

    def get_move(self, state):
        # list with sensible moves
        sensible_moves = [move for move in state.get_legal_moves(include_eyes=False)]
             
        
        #parcourt les coups possible et ressort celui qu'aurai jouer le policy network    
        if len(sensible_moves) > 0:

            move_probs = self.policy.eval_state(state, sensible_moves)  # A AJOUTER 
            max_prob = max(move_probs, key=itemgetter(1))
            return max_prob[0]
       
        # No 'sensible' moves available, so do pass move
        return go.PASS_MOVE
    

In [ ]:
# a mettre dans la classe 

## Chargement du modèle SL

* A FAIRE ajouter à CNN_SL de quoi charger un modele et ses poids

In [ ]:
filename="test.json"
policy_pl=CNN_policy.CNN()
policy_pl.load (filename) 
player=Player_pl(policy_pl)

In [ ]:
initial_weights=""
policy_pl.model.load_weights(args.initial_weights)
policy_op=CNN_policy.CNN()
policy_op = CNN_SL.load (file_name)  
policy_op.model.load_weights(args.initial_weights)

il faut compiler les modeles ?

In [ ]:
# pour l'apprentissage plus tard
from keras.optimizers import SGD
opti = SGD(lr=0.1)
player.policy.model.compile(optimizer=opti)

## Création des joueurs
-> la fonction get move renvoye le coup prédit par le SL en fct de l'état du jeu


In [38]:
player = Player_rd()
opponent =Player_rd()

## Déroulement des partie

In [39]:
import go
import time

In [40]:
#initialisation
nb_partie=10
taille=9
import features as ft
conv=ft.Preprocess()

In [48]:
#visualisation d'un gs
def vis_gs(gs):
    x=0
    taille=len(gs.board)
    while x < taille:
        y=0
        string=""
        while y < taille:
            case=gs.board[y,x] #  il faut inverser x et y ici
            if  case==-1:
                string+="W"
            if  case==0:
                string+="."
            if  case==1: 
                string+="B"
            string+=" "
            y+=1
        print string 
        x+=1
    if gs.current_player==1:
        print "current_player : black"
    if gs.current_player==-1:
        print "current_player : white"

In [49]:
# deroulement
start=time.time()
s1=0 #tps premiere partie

etat = [go.GameState(size=taille) for _ in range(nb_partie)] #liste des parties actuelle
coups=[[] for _ in range(nb_partie)]  #liste des coups joués
parties=[[] for _ in range(nb_partie)] # liste des etats du jeu
id_gagne=[] #liste des indices des parties gagnées
ratio=0

op_couleur="BLACK"
pl_couleur="WHite"
# on joue le premier coup de chaque partie
for i in range(nb_partie):
    coup=opponent.get_move(etat[i]) 
    coups[i].append(coup)
    etat[i].do_move(coup)
    parties[i].append(conv.state_to_tensor(etat[i]))

#on joue tout les coups
actuel=player
ancien=opponent
fin=0 # pour arreter la boucle
tour=1
while(fin<nb_partie): 
    
    for i in range(nb_partie):
            if(etat[i].is_end_of_game==False): # vérifie que la partie n'est pas fini
                coup = actuel.get_move(etat[i]) # on récupere le coup jpué 
                coups[i].append(coup) # on le sauvegarde
                etat[i].do_move(coup) #on le joue
                parties[i].append(conv.state_to_tensor(etat[i])) #on sauvegarde l'état du jeu
                
                if (i==1):
                    tour+=1
                    print
                    print("Coup numéro %i" %tour)
                    vis_gs(etat[i])
                    
                
                if(etat[i].is_end_of_game==True): 
                    fin+=1  # pour arreter la boucle
                    if(etat[i].get_winner()==-1): # -1 pour blanc
                        id_gagne.append(i)
                        ratio+=1
                        
                if (fin==1&s1==0):
                    print("1ere partie finie en %f secondes" % (time.time()-start))
                    s1=1
    #vis_gs(parties[1])
    

    #on change de joueur
    temp=actuel
    actuel=ancien
    ancien=actuel

ratio /=float(nb_partie)
print("%d executé en %f secondes"%(nb_partie,time.time()-start))
print("ratio de victoire: %f" % ratio)



1ere partie finie en 0.019654 secondes

Coup numéro 2
. . . . . . . W . 
. . . . . . . . . 
. . . . . . . . . 
. . . . . . . . . 
. . . . . . . . . 
. . . . . . . . . 
. B . . . . . . . 
. . . . . . . . . 
. . . . . . . . . 
current_player : black

Coup numéro 3
. . . . . . . W . 
. . . . . . . . . 
. . . . . . . . . 
. . . . . . . . . 
. . . . . . . . . 
. . . . . . . B . 
. B . . . . . . . 
. . . . . . . . . 
. . . . . . . . . 
current_player : white

Coup numéro 4
. . . . . . . W . 
. . . . . . . . . 
. . . . . . . . . 
. . . . . . . . . 
. . . . . . . . . 
. . . . . . . B . 
. B . W . . . . . 
. . . . . . . . . 
. . . . . . . . . 
current_player : black

Coup numéro 5
. . . . . . . W . 
. . . . . . . . . 
. . . . B . . . . 
. . . . . . . . . 
. . . . . . . . . 
. . . . . . . B . 
. B . W . . . . . 
. . . . . . . . . 
. . . . . . . . . 
current_player : white

Coup numéro 6
. . . . . . . W . 
. . . . . . . . . 
. . . . B . . . . 
. . . . . . . . . 
. . . . . . . . . 
. . . . . . . B

. B W W B . W W . 
. . B W . B W W W 
. B . W B B . W . 
current_player : black

Coup numéro 53
B . B B B B W W B 
. B W . B B B . . 
W B W . B W B . . 
. B . . . . W W . 
W B W W B . W W . 
W . . W . . . B W 
. B W W B . W W . 
. . B W . B W W W 
B B . W B B . W . 
current_player : white

Coup numéro 54
B . B B B B W W B 
. B W . B B B . . 
W B W . B W B . . 
. B . . . . W W . 
W B W W B . W W . 
W . . W . . . B W 
. B W W B . W W . 
. . B W W B W W W 
B B . W B B . W . 
current_player : black

Coup numéro 55
B . B B B B W W B 
. B W . B B B . . 
W B W . B W B . . 
. B . . . . W W . 
W B W W B B W W . 
W . . W . . . B W 
. B W W B . W W . 
. . B W W B W W W 
B B . W B B . W . 
current_player : white

Coup numéro 56
B . B B B B W W B 
. B W . B B B . . 
W B W . B W B . . 
. B . . . . W W . 
W B W W B B W W . 
W . . W . W . B W 
. B W W B . W W . 
. . B W W B W W W 
B B . W B B . W . 
current_player : black

Coup numéro 57
B . B B B B W W B 
. B W . B B B . . 
W B W . B . B . . 
. B . .

B . B B B B W W W 
B B B B B B B B W 
. B . B B B B B W 
B B B . . B W W W 
. B B B B B W W . 
B B B B B W W . W 
B B . W B W W W . 
B . B W W . W W W 
B B B W . . . W . 
current_player : white

Coup numéro 98
B . B B B B W W W 
B B B B B B B B W 
. B . B B B B B W 
B B B . W B W W W 
. B B B B B W W . 
B B B B B W W . W 
B B . W B W W W . 
B . B W W . W W W 
B B B W . . . W . 
current_player : black

Coup numéro 99
B . B B B B W W W 
B B B B B B B B W 
. B . B B B B B W 
B B B . W B W W W 
. B B B B B W W . 
B B B B B W W . W 
B B B W B W W W . 
B . B W W . W W W 
B B B W . . . W . 
current_player : white

Coup numéro 100
B . B B B B W W W 
B B B B B B B B W 
. B . B B B B B W 
B B B . W B W W W 
. B B B B B W W . 
B B B B B W W . W 
B B B W B W W W . 
B . B W W . W W W 
B B B W . W . W . 
current_player : black

Coup numéro 101
B . B B B B W W W 
B B B B B B B B W 
. B . B B B B B W 
B B B B . B W W W 
. B B B B B W W . 
B B B B B W W . W 
B B B W B W W W . 
B . B W W . W W W 
B B B 

In [44]:
# verif
id_gagne

[2, 5, 6, 8]

In [45]:
parties[2]

[array([[[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          ..., 
          [ 1.,  1.,  1., ...,  1.,  1.,  1.],
          [ 1.,  1.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.]],
 
         [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          ..., 
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.]],
 
         [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          ..., 
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.]],
 
         ..., 
         [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,

## Entrainement

In [ ]:
import keras.backend as K

for i in range(nb_partie):
    
    if i in id_gagne:
        # on apprend le modele
        opti.lr=K.abs(opti.lr) *1
        policy_pl.train_on_batch(parties[i],coups[i])  #a verifier
    else:
        #on dessaprend le modele --> donner des poids negatifs
        opti.lr=K.abs(opti.lr) *-1
        policy_pl.train_on_batch(parties[i],coups[i])  #a verifier